In [1]:
import cv2 as cv
import numpy as np

In [2]:
class cvDetector:
    # Empty function for slider arguments
    @staticmethod
    def nothing(x):
        pass
    # Takes min-max x and y coordinates
    # Returns center coordinates of given rectangle
    @staticmethod
    def rectangleCenter(xs, ys):
        xc = int((xs[0] + xs[1])/2)
        yc = int((ys[0] + ys[1])/2)
        obj_center_coordinate = (xc, yc)
        return obj_center_coordinate
    # Takes a frame (does not change the given argument frame)
    # Takes x's and y's in two ordered arrays
    # Returns a new frame with lines to given coordinates
    @staticmethod
    def drawLine(frame, x_list, y_list, line_colour = (0,0,255) , line_thickness = 1, font = cv.FONT_HERSHEY_SIMPLEX):
        frame_lined = frame.copy()
        center_x = int(frame_lined.shape[1]/2)
        center_y = int(frame_lined.shape[0]/2)
        center_coordinate = (center_x, center_y)
        for xs, ys in zip(x_list,y_list):
            obj_center_coordinate = cvDetector.rectangleCenter(xs, ys)
            line_lenght = ((obj_center_coordinate[0] -center_x)**2 + (obj_center_coordinate[1] -center_y)**2)**(1/2)
            cv.line(frame_lined, center_coordinate, obj_center_coordinate, line_colour, line_thickness)
            cv.putText(frame_lined,"Len: {:.2f}".format(line_lenght), (obj_center_coordinate[0],obj_center_coordinate[1]), font,
                                            1, (0, 0, 255), 2)
        return frame_lined
    # Takes a frame (does not change the given argument frame)
    # Takes contours from cv.findContours() method
    # Returns a new frame with rectangles on given contours
    # Returns drawn rectangles' min-max x coordinates and min-max y coordinates
    @staticmethod
    def drawRectangle(frame, contours, area_thresh=(0,50000), rectangle_color=(255,0,0), rectangle_thickness=2):
        area_l_thresh = area_thresh[0]
        area_u_thresh = area_thresh[1]
        frame_rectangled = frame.copy()
        x_list = []
        y_list = []
        for contour in contours:
            (x, y, w, h) = cv.boundingRect(contour)

            if cv.contourArea(contour) > area_l_thresh and cv.contourArea(contour) < area_u_thresh:
                cv.rectangle(frame_rectangled, (x, y), (x+w, y+h), rectangle_color, rectangle_thickness)
                cv.putText(frame_rectangled,"Status: {}".format("Detected"), (10,30), cv.FONT_HERSHEY_SIMPLEX,
                                        1, (0, 0, 255), 3)
                x_list.append((x,x+w))
                y_list.append((y,y+h))
        return frame_rectangled, x_list, y_list
    # Takes a frame (does not change the given argument frame)
    # Takes upper and lower bound for mask    
    @staticmethod
    def drawContours(frame,l_bound,u_bound):
        mask = cv.inRange(frame,l_bound,u_bound)
        frame_contoured = frame.copy()
        contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        cv.drawContours(frame_contoured, contours, -1, (255, 0, 0), 2)
        return frame_contoured
    # Takes barnames and and its min max positions of bars in "barnames" order
    @staticmethod
    def createTrackbars(barnames, windowname, gaps):
        for barname, gap in zip(barnames,gaps):
            cv.createTrackbar(barname, windowname, gap[0], gap[1], cvDetector.nothing)
    # Gets bar positions in "barnames" order
    @staticmethod
    def getTrackbars(barnames, windowname):
        pos_list = []
        for barname in barnames:
            pos_list.append(cv.getTrackbarPos(barname, windowname))
        return pos_list

In [3]:
vid_name = "bluedetection.mp4"

In [4]:
cap = cv.VideoCapture(vid_name)

In [5]:
filter_barnames = ['L_H','U_H',                         
                    'L_S','U_S',                         
                    'L_V','U_V',
                    'blur_flag', 'blur_k_size']
                                             
mask_barnames =  ['dil1_flag','dil1_iter', 
                    'erd1_flag','erd1_iter',   
                    'dil2_flag','dil2_iter', 
                    'erd2_flag','erd1_iter',     
                    'k_size_dil','k_size_erd']   
                    
draw_barnames = ['line_flag','cont_flag','rect_flag', 
                    'area_low','area_up']   

filter_gaps = [(0,180),(180,180),
                (0,255),(255,255),
                (0,255),(255,255),
                (0,1), (5,25)]
                
mask_gaps =  [(0,1),(0,10),
                (0,1),(0,10),
                (0,1),(0,10),
                (0,1),(0,10),
                (0,32),(0,32)]

draw_gaps = [(0,1),(0,1),(0,1),
            (0,1000), (10000,10000)]

In [6]:

cv.namedWindow("Filter",flags=0)
cv.namedWindow("Mask",flags=0)
cv.namedWindow("Draw",flags=0)


In [7]:
#Creating trackbars with given window names
cvDetector.createTrackbars(filter_barnames, "Filter", filter_gaps)
cvDetector.createTrackbars(mask_barnames, "Mask", mask_gaps)
cvDetector.createTrackbars(draw_barnames, "Draw", draw_gaps)
while(True):
    ret, frame = cap.read()
    if ret:
        # Converting frame to HSV
        frame_hsv = cv.cvtColor(frame,cv.COLOR_BGR2HSV)
        # Creating res_frame as copy of unprocessed frame
        res_frame = frame.copy()
        # Getting trackbar positions
        filter_pos_list = cvDetector.getTrackbars(filter_barnames, "Filter")
        mask_pos_list = cvDetector.getTrackbars(mask_barnames, "Mask")
        draw_pos_list = cvDetector.getTrackbars(draw_barnames, "Draw")
        # Assigning trackbar position values to relating variables
        # Effects on mask variables
        dilation1_flag = mask_pos_list[0]
        dilation1_iter = mask_pos_list[1]
        erosion1_flag = mask_pos_list[2]
        erosion1_iter = mask_pos_list[3]
        dilation2_flag = mask_pos_list[4]
        dilation2_iter = mask_pos_list[5]
        erosion2_flag = mask_pos_list[6]
        erosion2_iter = mask_pos_list[7]
        kernel_size_dilation = mask_pos_list[8]
        kernel_size_erosion = mask_pos_list[9]
        # Visualization variables
        line_flag = draw_pos_list[0]
        contour_flag = draw_pos_list[1]
        rectangle_flag = draw_pos_list[2]
        area_thresh_lower = draw_pos_list[3]
        area_thresh_upper = draw_pos_list[4]
        # Blur and filter variables
        blur_flag = filter_pos_list[6]
        blur_k_size = filter_pos_list[7]
        l_bound_hsv = np.array([filter_pos_list[0],filter_pos_list[2],filter_pos_list[4]])
        u_bound_hsv = np.array([filter_pos_list[1],filter_pos_list[3],filter_pos_list[5]])
        # Blur style
        if blur_flag:
            frame_hsv = cv.GaussianBlur(frame_hsv,(5,5),0)
        # Masking
        mask = cv.inRange(frame_hsv, l_bound_hsv, u_bound_hsv)
        # Dilation - Erosion kernels created
        kernel_dilation = np.ones(kernel_size_dilation, np.uint8)
        kernel_erosion = np.ones(kernel_size_erosion, np.uint8)
        # Mask refined created before applying effects on mask
        mask_refined = mask.copy()
        # Dilation - Erosion applying
        if dilation1_flag:
            mask_refined = cv.dilate(mask_refined, kernel_dilation, iterations=dilation1_iter)
        if erosion1_flag:
            mask_refined = cv.erode(mask_refined, kernel_erosion, iterations=erosion1_iter)
        if dilation2_flag:
            mask_refined = cv.dilate(mask_refined, kernel_dilation, iterations=dilation2_iter)
        if erosion2_flag:
            mask_refined = cv.erode(mask_refined, kernel_erosion, iterations=erosion2_iter)
        # Finding contours with final mask
        contours, _ = cv.findContours(mask_refined, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        # Drawing contours on frame
        if contour_flag:
            cv.drawContours(frame, contours, -1, (255, 0, 0), 2)
        # Drawing rectangles on frame an lines to the centers of rectangles
        if rectangle_flag:
            frame, x_list, y_list = cvDetector.drawRectangle(frame, contours, area_thresh= (area_thresh_lower, area_thresh_upper), rectangle_color=(255,0,0), rectangle_thickness=2)
            if line_flag:
                frame = cvDetector.drawLine(frame, x_list, y_list, line_colour = (0,0,255) , line_thickness = 1, font = cv.FONT_HERSHEY_SIMPLEX)
        # Finding result frame(res) with final mask and unprocessed frame 
        res = cv.bitwise_and(res_frame,res_frame,mask=mask_refined)
        # Showing processed frame, refined mask, result frame
        cv.imshow("frame",frame)
        cv.imshow("mask_refined",mask_refined)
        cv.imshow("res",res)
    else:
        cap.set(cv.CAP_PROP_POS_FRAMES, 0)
    key = cv.waitKey(24)
    if key==27:
        break
cap.release()
cv.destroyAllWindows()